In [1]:
import os
from glob import glob
import json
import pandas as pd
from tqdm import tqdm
from matplotlib import pyplot as plt
from IPython.display import Image, display
import time
import numpy as np
import cv2
from collections import defaultdict

In [2]:
scatter = pd.read_csv('/Users/minsukchang/Research/ChartQA/scatter/scatters.csv')


def real_corr(filename):
    direction, _, corr, exp = filename.split('_')
    row = scatter[(scatter['scatter_type'] == direction) & (scatter['corr'] == int(corr)) & (scatter['exp'] == int(exp))]
    if row.shape[0] == 0:
        print(filename)
    return row['corr_computed'].values[0]

In [3]:
def generate_base_components():
    return {
        "bubble": {
            "type": "react-component",
            "path": "phase2/assets/bubble.jsx",
            "response": [
                {
                    "id": "answer",
                    "prompt": "Recorded Answer:",
                    "required": True,
                    "location": "sidebar",
                    "type": "reactive"
                },
            ],
            "parameters": {
                "radius": 30,
                "blur_std": 13,
            },
            "instructionLocation": "belowStimulus",
            "nextButtonLocation": "belowStimulus"
        }
    }

In [4]:
def create_default_components():
    return {
        "welcome": {
            "type": "markdown",
            "path": "phase2/assets/welcome.md",
            "response": []
        },
        "consent": {
            "type": "markdown",
            "path": "phase2/assets/consent.md",
            "nextButtonText": "I agree",
            "response": []
        },
        "main_start": {
            "type": "markdown",
            "path": "phase2/assets/main_start.md",
            "response": []
        },
        "instruction": {
            "type": "markdown",
            "path": "phase2/assets/instruction.md",
            "response": []
        },
        "demographics": {
            "type": "markdown",
            "path": "phase2/assets/demographics.md",
            "response": [
                {
                    "id": "age",
                    "prompt": "What is your age?",
                    "required": False,
                    "location": "belowStimulus",
                    "type": "numerical",
                    "withDontKnow": True
                },
                {
                    "id": "gender",
                    "prompt": "What is your gender?",
                    "required": True,
                    "location": "belowStimulus",
                    "type": "dropdown",
                    "options": [
                        "Male",
                        "Female",
                        "Other",
                        "Prefer not to say"
                    ]
                }
            ]
        }
    }


def create_example_components():
    return {
        "bubble_example1": {
            "baseComponent": "bubble",
            "parameters": {
                "image": "https://raw.githubusercontent.com/jangsus1/ChartQA/main/scatter/pos_scatter_2_1.png",
                "example": True,
                "correlation": real_corr("pos_scatter_2_1")
            }
        },
        
    }


def create_main_components():
    components = {}
    for corr in [1, 3, 5, 7, 9]:
        for i in range(1, 7): # for phase 1
            for direction in ["pos", "neg"]:
                components[f"bubble_{corr}_{i}_{direction}"] = {
                    "baseComponent": "bubble",
                    "parameters": {
                        "image": f"https://raw.githubusercontent.com/jangsus1/ChartQA/main/scatter/{direction}_scatter_{corr}_{i}.png",
                        "example": False,
                        "correlation": real_corr(f"{direction}_scatter_{corr}_{i}")
                    }
                }
    return components


def sequence_generator(main_components, example_components):
    sequence = {
        "order": "fixed",
        "components": [
            "welcome",
            "consent",
            "demographics",
            "instruction",
            *example_components,
            "main_start",
            {
                "id": "images",
                "order": "latinSquare",
                "components": list(main_components.keys())
            }
        ]
    }
    return sequence

In [5]:
default_components = create_default_components()
main_components = create_main_components()
example_components = create_example_components()
components = default_components | main_components | example_components
sequence = sequence_generator(main_components, example_components)
baseComponents = generate_base_components()
print(f"Total number of components: {len(components)}")

Total number of components: 66


In [6]:
with open("config.json", "r") as f:
    config = json.load(f)
config['baseComponents'] = baseComponents
config['components'] = components
config['sequence'] = sequence

with open("config.json", "w") as f:
    json.dump(config, f, indent=4)